### Movie Rating Dataset: Movie Lens(2023)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
csv_path_ratings = '/content/drive/MyDrive/Grad/Data Mining/FinalProject/ml-32m/ratings.csv'
df_ratings = pd.read_csv(csv_path_ratings)

In [4]:
csv_path_movies = '/content/drive/MyDrive/Grad/Data Mining/FinalProject/ml-32m/movies.csv'
df_movies = pd.read_csv(csv_path_movies)

#### Review Dataset

We explore the review dataset from different user and the movie. One thing we can observe from the data is that, not all users will have reviewed all movies, in fact, each user only reviews certain movies. Using the limited reviews that a user has given, out goal is to predict movie review for the movies the user has not yet rated. If the predicted rating is high, we are more likely to recommend the user that movie.

In [5]:
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,17,4.0,944249077
1,1,25,1.0,944250228
2,1,29,2.0,943230976
3,1,30,5.0,944249077
4,1,32,5.0,943228858


In [6]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32000204 entries, 0 to 32000203
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 976.6 MB


#### Movie Dataset

We are going to be using this dataset to filter out movies from particular year of our interest (say 2023) in order to recommend.
- We dont want to recommend movie outside of that year because it would be unfeasible to run that unless we have really enormous compute.
- Also we want to recommend to user who has provided some review to these latest movies.  

In [7]:
df_movies.tail(20)

,movieId,title,genres
87565,292599,The Illusion of Abundance (2022),Documentary
87566,292601,Mountain Flesh (2022),Documentary
87567,292603,The Female Voice Of Afghanistan: Life After Li...,Documentary
87568,292605,Our River... Our Sky (2023),Drama|War
87569,292607,The Invention of the Other (2022),Documentary
87570,292609,The Wind Blows the Border (2022),Documentary
87571,292611,Kenya (2022),Documentary
87572,292613,Freelance (2023),Action|Comedy
87573,292615,Sparta (2022),Drama
87574,292617,Comeback (2023),Comedy|Drama


In [14]:
MOVIES_YEAR_OF_INTEREST='2023' # Select different year if you want to be recommended movie from any other year

In [21]:
df_movies_by_year=df_movies[df_movies['title'].str.contains(MOVIES_YEAR_OF_INTEREST)]
df_movies_by_year_count=df_movies_by_year.shape[0]
print(f"Movies Year ({MOVIES_YEAR_OF_INTEREST}) Count: ",df_movies_by_year_count )
df_movies_by_year.head(20)

Movies Year (2023) Count:  953


,movieId,title,genres
84230,281570,Asterix & Obelix: The Middle Kingdom (2023),Adventure|Children|Comedy
84254,281638,The Blue Caftan (2023),Drama|Romance
84330,281894,Bad Behaviour (2023),Comedy|Drama
84331,281896,Cassandro (2023),(no genres listed)
84333,281900,Eileen (2023),Drama|Mystery
84335,281904,Infinity Pool (2023),Horror|Sci-Fi|Thriller
84336,281906,Kim's Video (2023),Documentary
84337,281908,Kokomo City (2023),Documentary
84338,281910,Magazine Dreams (2023),Drama
84339,281912,Passages (2023),Drama


In [22]:
df_reviews_by_year=df_ratings[df_ratings['movieId'].isin(df_movies_by_year['movieId'])]

In [24]:
df_reviews_by_year_count=df_reviews_by_year.shape[0]
print(f"Reviews Year ({MOVIES_YEAR_OF_INTEREST}) Count: ",df_reviews_by_year_count )
df_reviews_by_year.head(20)

Reviews Year (2023) Count:  20788


,userId,movieId,rating,timestamp
5369,28,285593,3.0,1695439205
5370,28,286897,5.0,1692916119
5371,28,287699,5.0,1692916114
5372,28,288513,5.0,1693533442
5373,28,291485,4.0,1696391189
12298,79,287699,4.5,1690818303
12299,79,288513,4.5,1692977049
21588,144,284937,3.5,1693264555
26250,183,286901,4.0,1692105213
26251,183,286905,4.5,1692106157


### UV Factorization

In order to predict movie review for some selected user we plan to use UV Factorization techique. For this, we have 2 steps:
1. Create a matrix of user and rating.
2. Use algorithm for UV Factorization to come up with smaller matrices (U,V) with arbitrary latent features  
3. Predict the missing rating for all the users in the original matrix


##### Create a matrix of user and rating.

In [ ]:
# Pivot to create user × movie matrix with ratings
rating_matrix = df_ratings.pivot(index='userId', columns='movieId', values='rating')

print(rating_matrix)

<ipython-input-9-4ba64a60232d>:2: PerformanceWarning: The following operation may generate 16966441536 cells in the resulting pandas object.
  rating_matrix = df_ratings.pivot(index='userId', columns='movieId', values='rating')
